# 🎲 Projet : Simulateur Monte Carlo

## 🎯 Objectifs du projet

La **méthode de Monte Carlo** est une technique de simulation probabiliste fondamentale en :
- 📈 **Finance quantitative** : Pricing d'options, calcul de VaR, gestion de risque
- 🤖 **Machine Learning** : Bayesian optimization, reinforcement learning
- 🧮 **Calcul numérique** : Intégration, optimisation stochastique

Dans ce projet, vous allez implémenter 4 applications majeures :

1. ✅ **Estimation de π** : Introduction à Monte Carlo
2. ✅ **Marches aléatoires** : Simulation de prix d'actions (modèle Black-Scholes)
3. ✅ **Pricing d'options** : Valorisation d'options européennes
4. ✅ **Value at Risk (VaR)** : Mesure de risque de portefeuille

---

## 📚 Concepts théoriques

### Principe de Monte Carlo

1. **Générer** des échantillons aléatoires selon une distribution
2. **Calculer** une quantité d'intérêt pour chaque échantillon
3. **Agréger** les résultats (moyenne, quantiles, etc.)
4. **Converger** vers la vraie valeur quand nombre d'échantillons → ∞

### Loi des grands nombres

Si $X_1, X_2, ..., X_n$ sont i.i.d. avec $E[X_i] = \mu$ :

$$\frac{1}{n} \sum_{i=1}^{n} X_i \xrightarrow{n \to \infty} \mu$$

### Vitesse de convergence

Erreur standard : $\sigma / \sqrt{n}$

⚡ Pour diviser l'erreur par 2, il faut **4× plus d'échantillons** !

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pandas as pd
from typing import Tuple, List
from tqdm import tqdm

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
np.random.seed(42)

## 1️⃣ Estimation de π par Monte Carlo

### 🎯 Principe

1. Générer des points aléatoires dans un carré $[-1, 1] \times [-1, 1]$
2. Compter combien tombent dans le cercle de rayon 1 : $x^2 + y^2 \leq 1$
3. Ratio : $\frac{\text{points dans cercle}}{\text{points totaux}} \approx \frac{\pi r^2}{(2r)^2} = \frac{\pi}{4}$
4. Donc : $\pi \approx 4 \times \frac{\text{points dans cercle}}{\text{points totaux}}$

In [ ]:
def estimer_pi(n_points: int) -> Tuple[float, float]:
    """
    Estime π par Monte Carlo
    
    Returns:
        estimation: Valeur estimée de π
        erreur: Erreur absolue par rapport à np.pi
    """
    # Générer points aléatoires uniformes dans [-1, 1] × [-1, 1]
    x = np.random.uniform(-1, 1, n_points)
    y = np.random.uniform(-1, 1, n_points)
    
    # Distance au centre
    distances = x**2 + y**2
    
    # Points dans le cercle unitaire
    dans_cercle = distances <= 1
    
    # Estimation de π
    pi_estimate = 4 * np.sum(dans_cercle) / n_points
    erreur = abs(pi_estimate - np.pi)
    
    return pi_estimate, erreur

# Test avec différentes tailles d'échantillon
tailles = [100, 1000, 10000, 100000, 1000000]
resultats = []

print("🎲 Estimation de π par Monte Carlo\n")
print("="*60)
print(f"{'N points':<12} {'π estimé':<12} {'Erreur':<12} {'Erreur %':<12}")
print("="*60)

for n in tailles:
    pi_est, err = estimer_pi(n)
    resultats.append({'n': n, 'pi': pi_est, 'erreur': err})
    print(f"{n:<12,} {pi_est:<12.6f} {err:<12.6f} {err/np.pi*100:<12.4f}%")

print("="*60)
print(f"Vraie valeur : π = {np.pi:.10f}")

df_resultats = pd.DataFrame(resultats)

In [ ]:
# Visualisation
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Illustration graphique pour n=10000
ax1 = fig.add_subplot(gs[0, :])
n_vis = 10000
x_vis = np.random.uniform(-1, 1, n_vis)
y_vis = np.random.uniform(-1, 1, n_vis)
dans_cercle_vis = (x_vis**2 + y_vis**2) <= 1

ax1.scatter(x_vis[dans_cercle_vis], y_vis[dans_cercle_vis], 
           s=1, c='blue', alpha=0.5, label='Dans le cercle')
ax1.scatter(x_vis[~dans_cercle_vis], y_vis[~dans_cercle_vis], 
           s=1, c='red', alpha=0.5, label='Hors du cercle')

# Cercle théorique
theta = np.linspace(0, 2*np.pi, 1000)
ax1.plot(np.cos(theta), np.sin(theta), 'k-', linewidth=2, label='Cercle unitaire')
ax1.plot([-1, 1, 1, -1, -1], [-1, -1, 1, 1, -1], 'k--', linewidth=2, label='Carré')

pi_vis = 4 * np.sum(dans_cercle_vis) / n_vis
ax1.set_title(f'Méthode Monte Carlo : {n_vis:,} points → π ≈ {pi_vis:.4f}', 
             fontweight='bold', fontsize=14)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_aspect('equal')
ax1.legend(loc='upper right')
ax1.grid(alpha=0.3)

# 2. Convergence de l'estimation
ax2 = fig.add_subplot(gs[1, 0])
ax2.semilogx(df_resultats['n'], df_resultats['pi'], 'o-', linewidth=2, markersize=8)
ax2.axhline(np.pi, color='red', linestyle='--', linewidth=2, label='π vrai')
ax2.set_xlabel('Nombre de points')
ax2.set_ylabel('Estimation de π')
ax2.set_title('Convergence vers π', fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

# 3. Erreur en fonction de n
ax3 = fig.add_subplot(gs[1, 1])
ax3.loglog(df_resultats['n'], df_resultats['erreur'], 'o-', linewidth=2, markersize=8)
# Théorie : erreur ~ 1/√n
n_theory = np.array(tailles)
erreur_theory = 2 / np.sqrt(n_theory)  # Facteur 2 ajusté empiriquement
ax3.loglog(n_theory, erreur_theory, 'r--', linewidth=2, label='Théorique ~ 1/√n')
ax3.set_xlabel('Nombre de points')
ax3.set_ylabel('Erreur absolue')
ax3.set_title('Décroissance de l\'erreur', fontweight='bold')
ax3.legend()
ax3.grid(alpha=0.3)

# 4. Distribution des estimations (pour montrer la variance)
ax4 = fig.add_subplot(gs[1, 2])
n_repetitions = 1000
n_sample = 10000
estimations = [estimer_pi(n_sample)[0] for _ in range(n_repetitions)]

ax4.hist(estimations, bins=50, density=True, alpha=0.7, edgecolor='black')
ax4.axvline(np.pi, color='red', linestyle='--', linewidth=2, label=f'π = {np.pi:.4f}')
ax4.axvline(np.mean(estimations), color='green', linestyle='-', linewidth=2, 
           label=f'Moyenne = {np.mean(estimations):.4f}')
ax4.set_xlabel('Estimation de π')
ax4.set_ylabel('Densité')
ax4.set_title(f'Distribution ({n_repetitions} répétitions, n={n_sample:,})', 
             fontweight='bold')
ax4.legend()
ax4.grid(alpha=0.3)

# 5. Évolution au cours d'une simulation
ax5 = fig.add_subplot(gs[2, :])
n_max = 100000
x_evol = np.random.uniform(-1, 1, n_max)
y_evol = np.random.uniform(-1, 1, n_max)
dans_cercle_evol = (x_evol**2 + y_evol**2) <= 1

# Calcul cumulatif
cumsum_dans = np.cumsum(dans_cercle_evol)
n_points_evol = np.arange(1, n_max + 1)
pi_evol = 4 * cumsum_dans / n_points_evol

# Sous-échantillonner pour la visualisation
indices = np.logspace(0, np.log10(n_max), 1000, dtype=int) - 1
ax5.plot(n_points_evol[indices], pi_evol[indices], linewidth=1.5, alpha=0.8)
ax5.axhline(np.pi, color='red', linestyle='--', linewidth=2, label=f'π = {np.pi:.6f}')
ax5.fill_between(n_points_evol[indices], 
                 np.pi - 2/np.sqrt(n_points_evol[indices]), 
                 np.pi + 2/np.sqrt(n_points_evol[indices]), 
                 alpha=0.2, color='red', label='±2σ (approximatif)')
ax5.set_xscale('log')
ax5.set_xlabel('Nombre de points (échelle log)')
ax5.set_ylabel('Estimation de π')
ax5.set_title('Convergence progressive (simulation unique)', fontweight='bold')
ax5.legend()
ax5.grid(alpha=0.3)

plt.suptitle('Estimation de π par Monte Carlo', fontsize=16, fontweight='bold', y=0.995)
plt.show()

print(f"\n📊 Statistiques sur {n_repetitions} répétitions (n={n_sample:,}) :")
print(f"   Moyenne : {np.mean(estimations):.6f}")
print(f"   Écart-type : {np.std(estimations):.6f}")
print(f"   Erreur théorique : {1/np.sqrt(n_sample):.6f}")

## 2️⃣ Marches Aléatoires et Mouvement Brownien

### 📈 Modèle de Black-Scholes pour les prix d'actions

Le prix d'une action suit un **mouvement brownien géométrique** :

$$dS_t = \mu S_t dt + \sigma S_t dW_t$$

Où :
- $S_t$ : Prix au temps $t$
- $\mu$ : Rendement moyen (drift)
- $\sigma$ : Volatilité
- $W_t$ : Mouvement brownien standard

### Solution discrète (schéma d'Euler)

$$S_{t+\Delta t} = S_t \exp\left[(\mu - \frac{\sigma^2}{2})\Delta t + \sigma \sqrt{\Delta t} Z\right]$$

où $Z \sim \mathcal{N}(0, 1)$

In [ ]:
def simuler_prix_action(S0: float, mu: float, sigma: float, 
                       T: float, n_steps: int, n_simulations: int = 1) -> np.ndarray:
    """
    Simule des trajectoires de prix d'actions (modèle Black-Scholes)
    
    Parameters:
        S0: Prix initial
        mu: Rendement moyen annualisé
        sigma: Volatilité annualisée
        T: Horizon temporel (années)
        n_steps: Nombre de pas de temps
        n_simulations: Nombre de trajectoires à simuler
    
    Returns:
        prices: Array (n_simulations, n_steps+1) des prix simulés
    """
    dt = T / n_steps
    
    # Générer les chocs gaussiens
    Z = np.random.standard_normal((n_simulations, n_steps))
    
    # Calcul des rendements logarithmiques
    log_returns = (mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z
    
    # Calcul des prix par cumul des log-returns
    log_prices = np.zeros((n_simulations, n_steps + 1))
    log_prices[:, 0] = np.log(S0)
    log_prices[:, 1:] = np.log(S0) + np.cumsum(log_returns, axis=1)
    
    prices = np.exp(log_prices)
    
    return prices

# Paramètres d'une action type
S0 = 100  # Prix initial
mu = 0.08  # Rendement moyen 8% par an
sigma = 0.20  # Volatilité 20% par an
T = 1  # 1 an
n_steps = 252  # 252 jours de trading
n_sim = 1000

# Simulation
prix = simuler_prix_action(S0, mu, sigma, T, n_steps, n_sim)
temps = np.linspace(0, T, n_steps + 1)

In [ ]:
# Visualisation
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Quelques trajectoires individuelles
ax1 = axes[0, 0]
n_plot = 50
for i in range(n_plot):
    ax1.plot(temps, prix[i], alpha=0.5, linewidth=0.8)

ax1.plot(temps, np.mean(prix, axis=0), 'r-', linewidth=3, label='Moyenne')
ax1.axhline(S0, color='black', linestyle='--', linewidth=2, label=f'S₀ = {S0}')
ax1.set_xlabel('Temps (années)')
ax1.set_ylabel('Prix de l\'action')
ax1.set_title(f'{n_plot} trajectoires simulées', fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# 2. Distribution des prix finaux
ax2 = axes[0, 1]
prix_final = prix[:, -1]

ax2.hist(prix_final, bins=50, density=True, alpha=0.7, edgecolor='black', label='Simulation')

# Distribution théorique : log-normale
prix_range = np.linspace(prix_final.min(), prix_final.max(), 200)
mean_log = np.log(S0) + (mu - 0.5 * sigma**2) * T
std_log = sigma * np.sqrt(T)
pdf_theorique = stats.lognorm.pdf(prix_range, s=std_log, scale=np.exp(mean_log))

ax2.plot(prix_range, pdf_theorique, 'r-', linewidth=2, label='Théorique (log-normale)')
ax2.axvline(np.median(prix_final), color='green', linestyle='--', linewidth=2, 
           label=f'Médiane = {np.median(prix_final):.2f}')
ax2.axvline(np.mean(prix_final), color='orange', linestyle='--', linewidth=2,
           label=f'Moyenne = {np.mean(prix_final):.2f}')
ax2.set_xlabel('Prix final S(T)')
ax2.set_ylabel('Densité')
ax2.set_title(f'Distribution des prix à T={T} an', fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

# 3. Évolution de la moyenne et intervalle de confiance
ax3 = axes[1, 0]
moyenne_temp = np.mean(prix, axis=0)
std_temp = np.std(prix, axis=0)
percentile_5 = np.percentile(prix, 5, axis=0)
percentile_95 = np.percentile(prix, 95, axis=0)

ax3.plot(temps, moyenne_temp, 'b-', linewidth=2, label='Moyenne empirique')
# Moyenne théorique : E[S(t)] = S₀ exp(μt)
moyenne_theorique = S0 * np.exp(mu * temps)
ax3.plot(temps, moyenne_theorique, 'r--', linewidth=2, label='Moyenne théorique')

ax3.fill_between(temps, percentile_5, percentile_95, alpha=0.3, 
                label='Intervalle 90% (5%-95%)')
ax3.axhline(S0, color='black', linestyle=':', linewidth=1)
ax3.set_xlabel('Temps (années)')
ax3.set_ylabel('Prix')
ax3.set_title('Évolution moyenne et intervalle de confiance', fontweight='bold')
ax3.legend()
ax3.grid(alpha=0.3)

# 4. Distribution des rendements logarithmiques
ax4 = axes[1, 1]
log_returns_final = np.log(prix[:, -1] / S0)

ax4.hist(log_returns_final, bins=50, density=True, alpha=0.7, edgecolor='black', 
        label='Simulation')

# Distribution théorique : Normale
x_range = np.linspace(log_returns_final.min(), log_returns_final.max(), 200)
pdf_normal = stats.norm.pdf(x_range, loc=mean_log - np.log(S0), scale=std_log)
ax4.plot(x_range, pdf_normal, 'r-', linewidth=2, label='Théorique N(μ,σ²)')

ax4.axvline(np.mean(log_returns_final), color='green', linestyle='--', linewidth=2,
           label=f'Moyenne = {np.mean(log_returns_final):.4f}')
ax4.set_xlabel('Log-rendement ln(S(T)/S₀)')
ax4.set_ylabel('Densité')
ax4.set_title('Distribution des log-rendements', fontweight='bold')
ax4.legend()
ax4.grid(alpha=0.3)

plt.suptitle(f'Simulation de prix d\'actions (μ={mu*100:.0f}%, σ={sigma*100:.0f}%, T={T} an)', 
            fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Statistiques
print("\n📊 Statistiques des prix finaux :")
print(f"   Moyenne : {np.mean(prix_final):.2f} (théorique : {S0 * np.exp(mu * T):.2f})")
print(f"   Médiane : {np.median(prix_final):.2f} (théorique : {S0 * np.exp((mu - 0.5*sigma**2) * T):.2f})")
print(f"   Écart-type : {np.std(prix_final):.2f}")
print(f"   Min : {np.min(prix_final):.2f}")
print(f"   Max : {np.max(prix_final):.2f}")
print(f"\n   P(S(T) > S₀) = {np.mean(prix_final > S0)*100:.1f}%")
print(f"   P(S(T) > 1.2×S₀) = {np.mean(prix_final > 1.2*S0)*100:.1f}%")
print(f"   P(S(T) < 0.8×S₀) = {np.mean(prix_final < 0.8*S0)*100:.1f}%")

## 3️⃣ Pricing d'Options par Monte Carlo

### 📐 Option Européenne Call

**Payoff à maturité** :
$$C(T) = \max(S(T) - K, 0)$$

où $K$ est le strike (prix d'exercice).

**Prix de l'option** (formule de Black-Scholes) :
$$C(0) = e^{-rT} \mathbb{E}[\max(S(T) - K, 0)]$$

### 🎲 Algorithme Monte Carlo

1. Simuler $N$ trajectoires de prix jusqu'à $T$
2. Calculer le payoff pour chaque trajectoire : $C_i = \max(S_i(T) - K, 0)$
3. Estimer : $C(0) \approx e^{-rT} \frac{1}{N} \sum_{i=1}^{N} C_i$

### 📊 Option Put Européenne

$$P(T) = \max(K - S(T), 0)$$

In [ ]:
def pricer_option_monte_carlo(S0: float, K: float, r: float, sigma: float, 
                             T: float, option_type: str = 'call',
                             n_simulations: int = 100000) -> Tuple[float, float]:
    """
    Price une option européenne par Monte Carlo
    
    Parameters:
        S0: Prix initial de l'action
        K: Strike (prix d'exercice)
        r: Taux sans risque annualisé
        sigma: Volatilité annualisée
        T: Maturité (années)
        option_type: 'call' ou 'put'
        n_simulations: Nombre de simulations Monte Carlo
    
    Returns:
        prix: Prix de l'option
        std_error: Erreur standard de l'estimation
    """
    # Simuler les prix finaux
    # Utilisation de la formule exacte pour S(T)
    Z = np.random.standard_normal(n_simulations)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    
    # Calcul des payoffs
    if option_type == 'call':
        payoffs = np.maximum(ST - K, 0)
    elif option_type == 'put':
        payoffs = np.maximum(K - ST, 0)
    else:
        raise ValueError("option_type doit être 'call' ou 'put'")
    
    # Actualisation et calcul du prix
    prix = np.exp(-r * T) * np.mean(payoffs)
    
    # Erreur standard
    std_error = np.exp(-r * T) * np.std(payoffs) / np.sqrt(n_simulations)
    
    return prix, std_error

def prix_black_scholes(S0: float, K: float, r: float, sigma: float, 
                      T: float, option_type: str = 'call') -> float:
    """
    Prix analytique Black-Scholes (pour comparaison)
    """
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        prix = S0 * stats.norm.cdf(d1) - K * np.exp(-r * T) * stats.norm.cdf(d2)
    else:  # put
        prix = K * np.exp(-r * T) * stats.norm.cdf(-d2) - S0 * stats.norm.cdf(-d1)
    
    return prix

# Paramètres de l'option
S0 = 100  # Prix actuel de l'action
K = 105  # Strike
r = 0.05  # Taux sans risque 5%
sigma = 0.20  # Volatilité 20%
T = 1  # Maturité 1 an

print("📈 PRICING D'OPTIONS EUROPÉENNES\n")
print("="*70)
print(f"Paramètres : S₀={S0}, K={K}, r={r*100:.0f}%, σ={sigma*100:.0f}%, T={T} an\n")

# Pricing Call
prix_call_mc, err_call = pricer_option_monte_carlo(S0, K, r, sigma, T, 'call', n_simulations=100000)
prix_call_bs = prix_black_scholes(S0, K, r, sigma, T, 'call')

print("🔵 CALL OPTION (droit d'acheter à K)")
print(f"   Monte Carlo  : {prix_call_mc:.4f} ± {err_call:.4f}")
print(f"   Black-Scholes: {prix_call_bs:.4f}")
print(f"   Erreur       : {abs(prix_call_mc - prix_call_bs):.4f} ({abs(prix_call_mc - prix_call_bs)/prix_call_bs*100:.2f}%)\n")

# Pricing Put
prix_put_mc, err_put = pricer_option_monte_carlo(S0, K, r, sigma, T, 'put', n_simulations=100000)
prix_put_bs = prix_black_scholes(S0, K, r, sigma, T, 'put')

print("🔴 PUT OPTION (droit de vendre à K)")
print(f"   Monte Carlo  : {prix_put_mc:.4f} ± {err_put:.4f}")
print(f"   Black-Scholes: {prix_put_bs:.4f}")
print(f"   Erreur       : {abs(prix_put_mc - prix_put_bs):.4f} ({abs(prix_put_mc - prix_put_bs)/prix_put_bs*100:.2f}%)\n")

# Vérification de la parité put-call
# C - P = S₀ - K×exp(-rT)
parite_mc = prix_call_mc - prix_put_mc
parite_theorique = S0 - K * np.exp(-r * T)
print("✅ Vérification parité put-call : C - P = S₀ - K×e^(-rT)")
print(f"   Monte Carlo  : {parite_mc:.4f}")
print(f"   Théorique    : {parite_theorique:.4f}")
print(f"   Différence   : {abs(parite_mc - parite_theorique):.6f}")

In [ ]:
# Analyse de sensibilité et visualisations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Distribution des payoffs (Call)
ax1 = axes[0, 0]
n_sim_vis = 100000
Z = np.random.standard_normal(n_sim_vis)
ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
payoffs_call = np.maximum(ST - K, 0)

ax1.hist(payoffs_call, bins=100, density=True, alpha=0.7, edgecolor='black')
ax1.axvline(np.mean(payoffs_call), color='red', linestyle='--', linewidth=2,
           label=f'Payoff moyen = {np.mean(payoffs_call):.2f}')
ax1.set_xlabel('Payoff du Call')
ax1.set_ylabel('Densité')
ax1.set_title(f'Distribution des payoffs Call (K={K})', fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# 2. Distribution des payoffs (Put)
ax2 = axes[0, 1]
payoffs_put = np.maximum(K - ST, 0)

ax2.hist(payoffs_put, bins=100, density=True, alpha=0.7, edgecolor='black', color='orange')
ax2.axvline(np.mean(payoffs_put), color='red', linestyle='--', linewidth=2,
           label=f'Payoff moyen = {np.mean(payoffs_put):.2f}')
ax2.set_xlabel('Payoff du Put')
ax2.set_ylabel('Densité')
ax2.set_title(f'Distribution des payoffs Put (K={K})', fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

# 3. Payoff functions
ax3 = axes[0, 2]
S_range = np.linspace(50, 150, 200)
payoff_call_curve = np.maximum(S_range - K, 0)
payoff_put_curve = np.maximum(K - S_range, 0)

ax3.plot(S_range, payoff_call_curve, 'b-', linewidth=2, label='Call payoff')
ax3.plot(S_range, payoff_put_curve, 'r-', linewidth=2, label='Put payoff')
ax3.axvline(K, color='black', linestyle='--', alpha=0.5, label=f'Strike K={K}')
ax3.axvline(S0, color='green', linestyle=':', alpha=0.5, label=f'S₀={S0}')
ax3.set_xlabel('Prix de l\'action à maturité S(T)')
ax3.set_ylabel('Payoff')
ax3.set_title('Fonctions de payoff', fontweight='bold')
ax3.legend()
ax3.grid(alpha=0.3)

# 4. Prix en fonction du spot (moneyness)
ax4 = axes[1, 0]
spots = np.linspace(70, 130, 50)
prix_calls = [prix_black_scholes(s, K, r, sigma, T, 'call') for s in spots]
prix_puts = [prix_black_scholes(s, K, r, sigma, T, 'put') for s in spots]

ax4.plot(spots, prix_calls, 'b-', linewidth=2, label='Call')
ax4.plot(spots, prix_puts, 'r-', linewidth=2, label='Put')
ax4.axvline(K, color='black', linestyle='--', alpha=0.5, label=f'Strike K={K}')
ax4.set_xlabel('Prix spot S₀')
ax4.set_ylabel('Prix de l\'option')
ax4.set_title('Prix en fonction du spot (moneyness)', fontweight='bold')
ax4.legend()
ax4.grid(alpha=0.3)

# 5. Prix en fonction de la volatilité
ax5 = axes[1, 1]
sigmas = np.linspace(0.05, 0.50, 50)
prix_calls_vol = [prix_black_scholes(S0, K, r, sig, T, 'call') for sig in sigmas]
prix_puts_vol = [prix_black_scholes(S0, K, r, sig, T, 'put') for sig in sigmas]

ax5.plot(sigmas * 100, prix_calls_vol, 'b-', linewidth=2, label='Call')
ax5.plot(sigmas * 100, prix_puts_vol, 'r-', linewidth=2, label='Put')
ax5.axvline(sigma * 100, color='green', linestyle='--', alpha=0.5, 
           label=f'σ actuelle = {sigma*100:.0f}%')
ax5.set_xlabel('Volatilité σ (%)')
ax5.set_ylabel('Prix de l\'option')
ax5.set_title('Prix en fonction de la volatilité (vega)', fontweight='bold')
ax5.legend()
ax5.grid(alpha=0.3)

# 6. Prix en fonction de la maturité
ax6 = axes[1, 2]
maturites = np.linspace(0.1, 2, 50)
prix_calls_mat = [prix_black_scholes(S0, K, r, sigma, t, 'call') for t in maturites]
prix_puts_mat = [prix_black_scholes(S0, K, r, sigma, t, 'put') for t in maturites]

ax6.plot(maturites, prix_calls_mat, 'b-', linewidth=2, label='Call')
ax6.plot(maturites, prix_puts_mat, 'r-', linewidth=2, label='Put')
ax6.axvline(T, color='green', linestyle='--', alpha=0.5, label=f'T actuelle = {T} an')
ax6.set_xlabel('Maturité T (années)')
ax6.set_ylabel('Prix de l\'option')
ax6.set_title('Prix en fonction de la maturité (theta)', fontweight='bold')
ax6.legend()
ax6.grid(alpha=0.3)

plt.suptitle('Analyse de sensibilité des options européennes', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Observations :")
print("   • Call : valeur augmente avec S₀, σ, T")
print("   • Put : valeur augmente avec σ, T mais diminue avec S₀")
print("   • Volatilité ↑ ⇒ Options plus chères (plus d'incertitude)")
print("   • Maturité ↑ ⇒ Options plus chères (plus de temps pour bouger)")

## 4️⃣ Value at Risk (VaR) par Monte Carlo

### 📊 Définition de la VaR

La **Value at Risk** (VaR) au niveau de confiance $\alpha$ est la perte maximale sur un horizon donné, qui ne sera pas dépassée avec probabilité $\alpha$.

Mathématiquement :
$$\text{VaR}_\alpha(L) = -\inf\{l : P(L \leq l) \geq \alpha\}$$

où $L$ est la distribution des pertes.

### 🎯 VaR d'un portefeuille

Pour un portefeuille de $n$ actifs avec poids $w_1, ..., w_n$ :

1. Simuler les rendements des actifs (corrélés !)
2. Calculer le rendement du portefeuille : $R_p = \sum w_i R_i$
3. VaR = quantile $1-\alpha$ de la distribution des pertes

In [ ]:
def calculer_var_portefeuille(poids: np.ndarray, rendements_moyens: np.ndarray,
                             cov_matrix: np.ndarray, valeur_initiale: float,
                             horizon: int, niveau_confiance: float = 0.95,
                             n_simulations: int = 10000) -> dict:
    """
    Calcule la VaR et CVaR d'un portefeuille par Monte Carlo
    
    Parameters:
        poids: Poids des actifs dans le portefeuille
        rendements_moyens: Rendements moyens quotidiens de chaque actif
        cov_matrix: Matrice de covariance des rendements quotidiens
        valeur_initiale: Valeur initiale du portefeuille
        horizon: Horizon de temps (jours)
        niveau_confiance: Niveau de confiance (ex: 0.95 pour VaR 95%)
        n_simulations: Nombre de simulations Monte Carlo
    
    Returns:
        dict avec VaR, CVaR, et distributions
    """
    n_actifs = len(poids)
    
    # Simuler les rendements corrélés (distribution multivariée normale)
    rendements_simules = np.random.multivariate_normal(
        rendements_moyens * horizon,
        cov_matrix * horizon,
        size=n_simulations
    )
    
    # Rendement du portefeuille pour chaque simulation
    rendements_portefeuille = rendements_simules @ poids
    
    # Valeur finale du portefeuille
    valeurs_finales = valeur_initiale * (1 + rendements_portefeuille)
    
    # Pertes et gains
    PnL = valeurs_finales - valeur_initiale
    pertes = -PnL  # Convention : pertes positives
    
    # VaR : quantile des pertes
    VaR = np.percentile(pertes, niveau_confiance * 100)
    
    # CVaR (Expected Shortfall) : moyenne des pertes au-delà de la VaR
    CVaR = np.mean(pertes[pertes >= VaR])
    
    return {
        'VaR': VaR,
        'CVaR': CVaR,
        'pertes': pertes,
        'PnL': PnL,
        'rendements_portefeuille': rendements_portefeuille,
        'valeurs_finales': valeurs_finales
    }

# Exemple : Portefeuille de 3 actifs
print("📊 CALCUL DE LA VALUE AT RISK (VaR)\n")
print("="*70)

# Définition du portefeuille
actifs = ['Tech', 'Finance', 'Energie']
poids = np.array([0.4, 0.4, 0.2])  # 40% Tech, 40% Finance, 20% Energie
valeur_portefeuille = 1_000_000  # 1 million

# Rendements moyens quotidiens (annualisés / 252)
rendements_annuels = np.array([0.12, 0.08, 0.15])
rendements_moyens = rendements_annuels / 252

# Volatilités annuelles
volatilites_annuelles = np.array([0.25, 0.18, 0.30])
volatilites_quotidiennes = volatilites_annuelles / np.sqrt(252)

# Matrice de corrélation
corr_matrix = np.array([
    [1.0, 0.6, 0.3],   # Tech
    [0.6, 1.0, 0.4],   # Finance
    [0.3, 0.4, 1.0]    # Energie
])

# Matrice de covariance
cov_matrix = np.outer(volatilites_quotidiennes, volatilites_quotidiennes) * corr_matrix

print("Composition du portefeuille :")
for actif, w, r_ann, vol_ann in zip(actifs, poids, rendements_annuels, volatilites_annuelles):
    print(f"   {actif:10s} : {w*100:5.1f}% | Rendement: {r_ann*100:5.1f}% | Volatilité: {vol_ann*100:5.1f}%")

print(f"\nValeur du portefeuille : {valeur_portefeuille:,.0f} €")

# Calcul VaR pour différents horizons et niveaux de confiance
horizons = [1, 5, 10, 20]  # jours
niveaux_confiance = [0.95, 0.99]

print("\n" + "="*70)
print("VaR ET CVaR\n")

resultats_var = {}
for horizon in horizons:
    print(f"\n{'='*70}")
    print(f"Horizon : {horizon} jour(s)")
    print(f"{'='*70}")
    
    for niveau in niveaux_confiance:
        res = calculer_var_portefeuille(
            poids, rendements_moyens, cov_matrix, valeur_portefeuille,
            horizon, niveau, n_simulations=100000
        )
        
        resultats_var[(horizon, niveau)] = res
        
        print(f"\n🎯 Niveau de confiance : {niveau*100:.0f}%")
        print(f"   VaR  : {res['VaR']:>10,.0f} € ({res['VaR']/valeur_portefeuille*100:>5.2f}%)")
        print(f"   CVaR : {res['CVaR']:>10,.0f} € ({res['CVaR']/valeur_portefeuille*100:>5.2f}%)")
        print(f"   💡 Interprétation : Avec {niveau*100:.0f}% de confiance, la perte ne dépassera pas {res['VaR']:,.0f}€ en {horizon} jour(s)")
        print(f"      Si la perte dépasse la VaR, elle sera en moyenne de {res['CVaR']:,.0f}€ (CVaR)")

In [ ]:
# Visualisations complètes de la VaR
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Choisir un horizon pour les visualisations détaillées
horizon_vis = 10
niveau_vis = 0.95
res_vis = resultats_var[(horizon_vis, niveau_vis)]

# 1. Distribution des P&L avec VaR et CVaR
ax1 = fig.add_subplot(gs[0, :])
ax1.hist(res_vis['PnL'], bins=100, density=True, alpha=0.7, edgecolor='black')
ax1.axvline(0, color='black', linestyle='-', linewidth=2, label='Break-even')
ax1.axvline(-res_vis['VaR'], color='red', linestyle='--', linewidth=2, 
           label=f"VaR {niveau_vis*100:.0f}% = {res_vis['VaR']:,.0f}€")
ax1.axvline(-res_vis['CVaR'], color='darkred', linestyle='--', linewidth=2,
           label=f"CVaR = {res_vis['CVaR']:,.0f}€")

# Zone au-delà de la VaR
pertes_extremes = res_vis['PnL'][res_vis['PnL'] < -res_vis['VaR']]
ax1.hist(pertes_extremes, bins=30, density=True, alpha=0.5, color='red', edgecolor='darkred')

ax1.set_xlabel('P&L (€)')
ax1.set_ylabel('Densité')
ax1.set_title(f'Distribution des P&L - Horizon {horizon_vis} jours', fontweight='bold', fontsize=14)
ax1.legend()
ax1.grid(alpha=0.3)

# Annotations
prob_perte = np.mean(res_vis['PnL'] < 0) * 100
prob_gain = np.mean(res_vis['PnL'] > 0) * 100
ax1.text(0.02, 0.95, f'P(Perte) = {prob_perte:.1f}%\nP(Gain) = {prob_gain:.1f}%',
        transform=ax1.transAxes, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8), fontsize=11)

# 2. VaR en fonction du niveau de confiance
ax2 = fig.add_subplot(gs[1, 0])
niveaux = np.linspace(0.90, 0.99, 20)
vars_niveaux = [np.percentile(res_vis['pertes'], n*100) for n in niveaux]

ax2.plot(niveaux * 100, np.array(vars_niveaux) / 1000, 'o-', linewidth=2, markersize=6)
ax2.axhline(res_vis['VaR'] / 1000, color='red', linestyle='--', alpha=0.5)
ax2.set_xlabel('Niveau de confiance (%)')
ax2.set_ylabel('VaR (milliers €)')
ax2.set_title('VaR vs Niveau de confiance', fontweight='bold')
ax2.grid(alpha=0.3)

# 3. VaR en fonction de l'horizon
ax3 = fig.add_subplot(gs[1, 1])
horizons_plot = [1, 5, 10, 15, 20]
vars_95 = [resultats_var.get((h, 0.95), {'VaR': 0})['VaR'] / 1000 
           for h in horizons_plot if (h, 0.95) in resultats_var]
vars_99 = [resultats_var.get((h, 0.99), {'VaR': 0})['VaR'] / 1000 
           for h in horizons_plot if (h, 0.99) in resultats_var]

horizons_existants = [h for h in horizons_plot if (h, 0.95) in resultats_var]

ax3.plot(horizons_existants, vars_95, 'o-', linewidth=2, markersize=8, label='VaR 95%')
ax3.plot(horizons_existants, vars_99, 's-', linewidth=2, markersize=8, label='VaR 99%')
ax3.set_xlabel('Horizon (jours)')
ax3.set_ylabel('VaR (milliers €)')
ax3.set_title('VaR vs Horizon temporel', fontweight='bold')
ax3.legend()
ax3.grid(alpha=0.3)

# 4. CVaR vs VaR
ax4 = fig.add_subplot(gs[1, 2])
horizons_cv = [h for h in horizons if (h, 0.95) in resultats_var]
vars_cv = [resultats_var[(h, 0.95)]['VaR'] / 1000 for h in horizons_cv]
cvars_cv = [resultats_var[(h, 0.95)]['CVaR'] / 1000 for h in horizons_cv]

x_cv = np.arange(len(horizons_cv))
width = 0.35

ax4.bar(x_cv - width/2, vars_cv, width, label='VaR 95%', alpha=0.8)
ax4.bar(x_cv + width/2, cvars_cv, width, label='CVaR', alpha=0.8)
ax4.set_xlabel('Horizon (jours)')
ax4.set_ylabel('Montant (milliers €)')
ax4.set_title('VaR vs CVaR (Expected Shortfall)', fontweight='bold')
ax4.set_xticks(x_cv)
ax4.set_xticklabels([f'{h}j' for h in horizons_cv])
ax4.legend()
ax4.grid(axis='y', alpha=0.3)

# 5. Distribution des rendements du portefeuille
ax5 = fig.add_subplot(gs[2, 0])
ax5.hist(res_vis['rendements_portefeuille'] * 100, bins=100, 
        density=True, alpha=0.7, edgecolor='black')

# Fit normal
mu_r = np.mean(res_vis['rendements_portefeuille']) * 100
sigma_r = np.std(res_vis['rendements_portefeuille']) * 100
x_r = np.linspace(res_vis['rendements_portefeuille'].min() * 100, 
                  res_vis['rendements_portefeuille'].max() * 100, 200)
ax5.plot(x_r, stats.norm.pdf(x_r, mu_r, sigma_r), 'r-', linewidth=2, 
        label=f'N({mu_r:.2f}%, {sigma_r:.2f}%)')

ax5.axvline(0, color='black', linestyle='--', linewidth=1)
ax5.set_xlabel('Rendement du portefeuille (%)')
ax5.set_ylabel('Densité')
ax5.set_title(f'Distribution des rendements ({horizon_vis}j)', fontweight='bold')
ax5.legend()
ax5.grid(alpha=0.3)

# 6. Q-Q plot pour vérifier la normalité
ax6 = fig.add_subplot(gs[2, 1])
stats.probplot(res_vis['rendements_portefeuille'], dist="norm", plot=ax6)
ax6.set_title('Q-Q Plot (test de normalité)', fontweight='bold')
ax6.grid(alpha=0.3)

# 7. Évolution temporelle (exemple de trajectoires)
ax7 = fig.add_subplot(gs[2, 2])
# Simuler quelques trajectoires complètes
n_traj = 100
valeurs_traj = np.zeros((n_traj, horizon_vis + 1))
valeurs_traj[:, 0] = valeur_portefeuille

for t in range(1, horizon_vis + 1):
    rendements_jour = np.random.multivariate_normal(rendements_moyens, cov_matrix, n_traj)
    rendements_p = rendements_jour @ poids
    valeurs_traj[:, t] = valeurs_traj[:, t-1] * (1 + rendements_p)

temps_traj = np.arange(horizon_vis + 1)
for i in range(n_traj):
    ax7.plot(temps_traj, valeurs_traj[i] / 1000, alpha=0.3, linewidth=0.8)

ax7.plot(temps_traj, np.mean(valeurs_traj, axis=0) / 1000, 'r-', 
        linewidth=3, label='Moyenne')
ax7.axhline(valeur_portefeuille / 1000, color='black', linestyle='--', 
           linewidth=2, label='Valeur initiale')
ax7.set_xlabel('Temps (jours)')
ax7.set_ylabel('Valeur du portefeuille (milliers €)')
ax7.set_title(f'{n_traj} trajectoires simulées', fontweight='bold')
ax7.legend()
ax7.grid(alpha=0.3)

plt.suptitle(f'Analyse complète de la Value at Risk - Portefeuille {valeur_portefeuille:,.0f}€', 
            fontsize=16, fontweight='bold', y=0.995)
plt.show()

# Statistiques finales
print("\n" + "="*70)
print("📊 STATISTIQUES GLOBALES\n")
print(f"Rendement moyen attendu ({horizon_vis}j) : {np.mean(res_vis['PnL']):>10,.0f} €")
print(f"Écart-type des P&L ({horizon_vis}j)      : {np.std(res_vis['PnL']):>10,.0f} €")
print(f"Pire perte observée                       : {np.min(res_vis['PnL']):>10,.0f} €")
print(f"Meilleur gain observé                     : {np.max(res_vis['PnL']):>10,.0f} €")
print(f"\nRatio Sharpe (simplifié)                  : {np.mean(res_vis['rendements_portefeuille']) / np.std(res_vis['rendements_portefeuille']):.4f}")

## 🎯 Conclusion du Projet

### ✅ Ce que vous avez appris

1. **Méthode Monte Carlo** : Principe, convergence, erreur standard
2. **Mouvement brownien géométrique** : Modélisation de prix d'actions
3. **Pricing d'options** : Valorisation par simulation vs formule analytique
4. **Value at Risk** : Mesure de risque de portefeuille avec corrélations

### 🤖 Applications en ML et Finance

| Technique | ML | Finance |
|-----------|----|---------|
| Monte Carlo | Bayesian optimization, MCMC | Pricing, Risk management |
| Marches aléatoires | Reinforcement learning | Market simulation |
| Sampling | Data augmentation, GANs | Scenario generation |
| Quantiles | Confidence intervals | VaR, stress testing |

### 📚 Pour aller plus loin

- **Options exotiques** : Asiatiques, lookback, barrier options
- **Réduction de variance** : Variables antithétiques, importance sampling
- **Calibration** : Volatilité implicite, surface de volatilité
- **Backtesting** : Validation des modèles de VaR

---

**Félicitations ! 🎉 Vous maîtrisez maintenant la simulation Monte Carlo pour la finance quantitative !**